# Simple Test Case for Suppressor

The equations can be simplified to

$c_{s,xx} = 0$

$-c_{s,x} = c_s \left( 1 - \theta \right)$

$\dot{\theta} = c_s \left( 1 - \theta \right) - v \theta$

if we assume that a lot of the parameters 1. We have a single parameter $v$. The solution to this is

$ c^2 + c \left( 2 v - 1 \right) -v = 0$

and

$\theta = \frac{c}{v + c}$

In [1]:
from main import run, eta_func, calc_eta
from toolz import pipe
from scipy import roots
from toolz_ import rcompose
import numpy as np


def get_params():
    return dict(
        diff_sup=1.0,
        sup_inf=1.0,
        delta=1.0,
        k_plus=1.0,
        k_minus=1.0,
        gamma=1.0,
        dt=1e+10,
        nx=1000,
        max_steps=1,
        max_sweeps=15,
        v0=1.0,
        vm=0.0,
        output=False,
        alpha=0.4,
        F=96485.332,
        R=8.314,
        T=270.0,
        theta_ini=0.0,
        sup_ini=0.0,
        tf=1.
    )

    
def jj(params):
    return 1. / eta_func(params, params['v0']) 


def update_params(vel):
    return pipe(
        get_params(),
        lambda x: dict(x, j0=jj(x) * vel, j1=jj(x) * vel)
    )


def answer(vel):
    return pipe(
        max(roots([1, 2 * vel - 1, -vel])),
        lambda x: (x, x / (x + vel))
    )


predict = rcompose(
    update_params,
    run,
    lambda x: (x['sup'][0], x['theta']['new'])
)


def check(vel):
    return np.allclose(answer(vel), predict(vel), rtol=1e-3)


assert check(0.1)
assert check(0.5)
assert check(0.25)
assert check(0.75)
assert check(100.0)
